In [1]:
from harp.clock import decode_harp_clock
from open_ephys.analysis import Session
import datetime
from aind_ephys_rig_qc.temporal_alignment import search_harp_line
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import os
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from scipy.io import loadmat
from scipy.stats import zscore

from open_ephys.analysis import Session
from pathlib import Path
import glob

import spikeinterface as si
import xarray as xr
import json
import seaborn as sns
from PyPDF2 import PdfMerger
from aind_ephys_utils import align 
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import re
from aind_dynamic_foraging_basic_analysis.lick_analysis import load_nwb, load_data
from hdmf_zarr.nwb import NWBZarrIO
from beh_functions import session_dirs, parseSessionID, load_model_dv, makeSessionDF

%matplotlib inline
%matplotlib widget

### Load behavior and model data

In [2]:
session_id = 'behavior_716325_2024-05-31_10-31-14'
model_name = 'stan_qLearning_5params'

In [3]:
session_dir = session_dirs(session_id, model_name)
nwb = load_nwb(session_dir['nwb_dir'])
model_dv, session_cut = load_model_dv(session_id, model_name)
trial_df = makeSessionDF(nwb, cut=session_cut)
session_df = pd.merge(trial_df, model_dv, left_index=True, right_index=True, suffixes=('', '_model'))
for column in session_df.columns:
    if '_model' in column:
        session_df.drop(column, axis=1, inplace=True)
session_df.head()

,outcome,choice,laser,outcome_prev,laser_prev,choices_prev,go_cue_time,choice_time,outcome_time,Q_r,Q_l,pe,pChoice
0,1.0,1.0,0.0,NaN,NaN,NaN,3.892971e+06,3.892972e+06,3.892972e+06,0.000000,0.0,1.000000,0.297878
1,1.0,1.0,0.0,1.0,0.0,1.0,3.892984e+06,3.892985e+06,3.892985e+06,0.836632,0.0,0.163368,0.899718
2,1.0,1.0,0.0,1.0,0.0,1.0,3.893001e+06,3.893001e+06,3.893001e+06,0.960854,0.0,0.039146,0.937385
3,1.0,1.0,0.0,1.0,0.0,1.0,3.893035e+06,3.893035e+06,3.893035e+06,0.988901,0.0,0.011099,0.943527
4,0.0,1.0,0.0,1.0,0.0,1.0,3.893107e+06,3.893108e+06,3.893108e+06,0.996498,0.0,-0.996498,0.944997


### Load neuron data, realign if needed

In [8]:
# detect if alignment timestamps exist
units = nwb.units

In [27]:
unit_spikes = units[:][['ks_unit_id', 'spike_times']]
unit_qc = units[:][['ks_unit_id', 'isi_violations_ratio', 'firing_rate', 'presence_ratio', 'amplitude_cutoff', 'decoder_label']]
unit_wf = units[:][['ks_unit_id', 'waveform_mean', 'waveform_sd']]

### Infer major drift point of all units pass qc and cut session short

### Plot all units pass qc

### Plot all opto tagged units